In [2]:
import xgboost as xgb
import pandas as pd
import model_helper_functions as mod
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error,  r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [3]:
df =  pd.read_parquet("all_cleaned_data_augmented.parquet")
df = mod.necessary_fields(df)
df['Hour'] = df['DropoffDatetime'].apply(mod.round_time_to_int)
df.head()

,PickupDatetime,DropoffDatetime,TripDuration,TripDistance,PULocationID,DOLocationID,FareAmount,TipAmount,NextPU,Hour
0,2023-10-01 00:57:33,2023-10-01 01:07:58,10.416667,1.45,166.0,74.0,12.1,2.92,74,1
1,2023-10-01 01:00:16,2023-10-01 01:06:13,5.950000,0.89,74.0,42.0,7.9,0.00,82,1
2,2023-10-01 00:51:52,2023-10-01 01:00:32,8.666667,2.38,83.0,129.0,13.5,0.00,116,1
3,2023-10-01 00:03:39,2023-10-01 00:11:20,7.683333,2.26,74.0,263.0,11.4,3.33,7,0
4,2023-10-01 00:27:42,2023-10-01 00:39:10,11.466667,2.14,74.0,236.0,13.5,2.81,80,1


In [4]:

# X = df.fillna(-999)

y = df["NextPU"]

X = df.drop(columns = ['NextPU', 'DropoffDatetime', 'PickupDatetime'])

le = LabelEncoder()
encoded_y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, encoded_y, test_size=0.1, random_state=42, stratify=y)

num_classes = len(np.unique(train_y))

print('X, y generated')



X, y generated


In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=num_classes,
    eval_metric = 'mlogloss',
    use_label_encoder=False,
    n_estimators = 500, 
    max_depth = 4, 
    eta = 0.1, 
    subsample = 0.1, 
    colsample_bytree = 1.0,
    random_state=42,
    verbosity=1
) # Need to fine tune all these parameters

from tqdm import tqdm

from xgboost.callback import TrainingCallback

class TQDMProgressBar(TrainingCallback):
    def __init__(self, total):
        self.pbar = tqdm(total=total, desc="Training")
    
    def after_iteration(self, model, epoch, evals_log):
        self.pbar.update(1)
        return False  # return True to stop training early

    def after_training(self, model):
        self.pbar.close()

dtrain = xgb.DMatrix(train_X, label=train_y)
dvalid = xgb.DMatrix(test_X, label=test_y)

params = {
    'objective': 'multi:softmax',
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
}

booster = xgb.train(
    params,
    dtrain,
    num_boost_round=num_rounds,
    evals=[(dvalid, "eval")],
    early_stopping_rounds=20,
    callbacks=[TQDMProgressBar(total=num_rounds)]
)

In [ ]:
model.fit(
    train_X, 
    train_y,
    eval_set=[(test_X, test_y)],
    verbose=True,
    early_stopping_rounds=20,
)

In [9]:
pred_y = booster.predict(test_X)

print(r2_score(test_y, pred_y))
print(root_mean_squared_error(test_y, pred_y))

NameError: name 'booster' is not defined